# 集中趋势测量
By Evgenia "Jenny" Nitishinskaya, Maxwell Margenot, and Delaney Mackenzie.

Part of the Quantopian Lecture Series:

* [www.quantopian.com/lectures](https://www.quantopian.com/lectures)
* [github.com/quantopian/research_public](https://github.com/quantopian/research_public)


---

在这节课中，我们将讨论如何使用单个数字来总结一组数据。目标是捕获关于数据分布的信息。

# 算术平均值

算术平均数经常被用来总结数字数据，通常被认为是“平均数”的意思。它的定义是观测值之和除以观测值的个数:
$$\mu = \frac{\sum_{i=1}^N X_i}{N}$$

其中$X_1, X_2, \ldots , X_N$是我们的观察值。

In [1]:
# Two useful statistical libraries
import scipy.stats as stats
import numpy as np

# We'll use these two data sets as examples
x1 = [1, 2, 2, 3, 4, 5, 5, 7]
x2 = x1 + [100]

print('Mean of x1:', sum(x1), '/', len(x1), '=', np.mean(x1)) 
print( 'Mean of x2:', sum(x2), '/', len(x2), '=', np.mean(x2))

Mean of x1: 29 / 8 = 3.625
Mean of x2: 129 / 9 = 14.333333333333334


我们还可以定义一个<i>加权的</i>算术平均值，这对于明确指定每个观测值应该被计数的次数很有用。例如，在计算一个投资组合的平均价值时，说你70%的股票是X型股票要比列出你持有的所有股票更方便。

加权算术平均值定义为
$$\sum_{i=1}^n w_i X_i $$

其中 $\sum_{i=1}^n w_i = 1$.在通常的算术平均值中，对于所有的$i$我们有$w_i = 1/n$。

# 中位数

一组数据的中位数是按递增或递减顺序排序时出现在列表中间的数字。当我们有一个奇数$n$的数据点时，这就是位置$(n+1)/2$的值。当我们有偶数个数据点时，列表分成两半，中间没有项目;因此，我们将中位数定义为位置$n/2$和$(n+2)/2$的值的平均值。

与算术平均值相比，中位数受数据极值的影响较小。它告诉我们将数据集一分为二的值，而不是其他值的大小。

In [2]:
print('Median of x1:', np.median(x1)) 
print('Median of x2:', np.median(x2)) 

Median of x1: 3.5
Median of x2: 4.0


# 众数

众数是数据集中最常出现的值。它可以应用于非数值数据，不像平均值和中位数。它有用的一种情况是数据的可能值是独立的。例如，在一个加权骰子的结果中，出现6并不意味着它可能出现5;所以知道数据集的众数是6比知道它的平均值是4.5更有用.

In [6]:
# Scipy has a built-in mode function, but it will return exactly one value even if two values occur the same number of times, or if no value appears more than once
print ('One mode of x1:', stats.mode(x1)[0][0])

# So we will write our own
def mode(l):
    # Count the number of times each element appears in the list
    counts = {}
    for e in l:
        if e in counts:
            counts[e] += 1
        else:
            counts[e] = 1
            
    # Return the elements that appear the most times
    maxcount = 0
    modes = {}
    for (key, value) in counts.items():
        if value > maxcount:
            maxcount = value
            modes = {key}
        elif value == maxcount:
            modes.add(key)
            
    if maxcount > 1 or len(l) == 1:
        return list(modes)
    return 'No mode'
    
print ('All of the modes of x1:', mode(x1))

One mode of x1: ModeResult(mode=array([2]), count=array([2]))
All of the modes of x1: [2, 5]


对于可以接受许多不同值的数据(如返回数据)，可能不会有任何值出现超过一次。在这种情况下，我们可以存储值，就像我们在构建直方图时所做的那样，然后找到数据集的众数，其中每个值都用其存储的名称替换。也就是说，我们找出哪些bin元素最常落入其中。

In [7]:
import pandas_datareader.data as data
# Get return data for an asset and compute the mode of the data set
start = '2014-01-01'
end = '2015-01-01'
pricing = data.get_data_yahoo('SPY',start, end)['Close']
returns = pricing.pct_change()[1:]
print ('Mode of returns:', mode(returns))

# Since all of the returns are distinct, we use a frequency distribution to get an alternative mode.
# np.histogram returns the frequency distribution over the bins as well as the endpoints of the bins
hist, bins = np.histogram(returns, 20) # Break data up into 20 bins
maxfreq = max(hist)
# Find all of the bins that are hit with frequency maxfreq, then print the intervals corresponding to them
print ('Mode of bins:', [(bins[i], bins[i+1]) for i, j in enumerate(hist) if j == maxfreq])

Mode of returns: No mode
Mode of bins: [(-0.0012499981123169877, 0.0011117022955209332)]


# 几何平均数

算术平均数用加法，几何平均数用乘法:
$$ G = \sqrt[n]{X_1X_1\ldots X_n} $$

对于观测值$X_i \geq 0$。我们也可以用对数把它改写成算术平均数:
$$ \ln G = \frac{\sum_{i=1}^n \ln X_i}{n} $$

几何平均值总是小于或等于算术平均值(当处理非负观测值时)，只有当所有观测值相同时才相等.

In [8]:
# Use scipy's gmean function to compute the geometric mean
print('Geometric mean of x1:', stats.gmean(x1) )
print('Geometric mean of x2:', stats.gmean(x2)) 

Geometric mean of x1: 3.0941040249774403
Geometric mean of x2: 4.552534587620071


如果我们想计算几何平均值当我们有负的观测值时呢?这个问题在资产回报的情况下很容易解决，我们的值总是至少为 $-1$。我们可以在回报率$R_t$上加1，得到$1 + R_t$，它是连续两个时间段内资产价格的比率(而不是价格之间变化的百分比，$R_t$)。这个量总是非负的。我们可以计算几何平均回报，
$$ R_G = \sqrt[T]{(1 + R_1)\ldots (1 + R_T)} - 1$$

In [9]:
# Add 1 to every value in the returns array and then compute R_G
ratios = returns + np.ones(len(returns))
R_G = stats.gmean(ratios) - 1
print( 'Geometric mean of returns:', R_G)

Geometric mean of returns: 0.00042454196714802883


几何平均是这样定义的，如果整个时间段的收益率是常数，并且等于$R_G$，那么证券的最终价格将与收益率 $R_1, \ldots, R_T$相同.

In [10]:
T = len(returns)
init_price = pricing[0]
final_price = pricing[T]
print('Initial price:', init_price) 
print('Final price:', final_price) 
print('Final price as computed with R_G:', init_price*(1 + R_G)**T) 

Initial price: 184.69000244140625
Final price: 205.5399932861328
Final price as computed with R_G: 205.53999328613105


# 调和平均数

调和平均数比其他平均数使用得少。它被定义为
$$ H = \frac{n}{\sum_{i=1}^n \frac{1}{X_i}} $$

和几何平均值一样，我们可以把调和平均值改写成算术平均值。调和平均数的倒数是观测倒数的算术平均数:
$$ \frac{1}{H} = \frac{\sum_{i=1}^n \frac{1}{X_i}}{n} $$

非负数$X_i$的调和平均数最多是几何平均数(最多是算术平均数)，只有当所有的观察值相等时它们才相等。

In [11]:
print('Harmonic mean of x1:', stats.hmean(x1)) 
print('Harmonic mean of x2:', stats.hmean(x2)) 

Harmonic mean of x1: 2.5590251332825593
Harmonic mean of x2: 2.869723656240511


当数据可以自然地用比率表示时，就可以使用调和平均数。例如，在平均成本投资策略中，固定数额的资金会定期用于购买股票。股价越高，遵循这种策略的投资者购买的股票就越少。他们购买股票的平均(算术平均数)是股票价格的调和平均数。

# 点估算可能具有欺骗性

本质上均值隐藏了大量信息，因为它们将整个分布压缩成一个数字。因此，经常使用一个数字的“点估计”或度量，可能会在您的数据中隐藏大问题。你应该小心地确保你在汇总数据时没有丢失关键信息，并且你应该很少(如果有的话)在使用平均值时不同时参考一个传播度量。

## 潜在的分布可能是错误的

即使您使用正确的平均值和传播度量，如果您的底层分布不是您所认为的那样，那么它们也没有任何意义。例如，使用标准偏差来度量事件的频率通常会假定为正态。除非必须，否则不要假设分布，在这种情况下，您应该严格检查数据是否符合您假设的分布。

## 参考
* "Quantitative Investment Analysis", by DeFusco, McLeavey, Pinto, and Runkle